In [2]:
import pandas as pd
import numpy as np
import sys, os
if not os.path.exists('OSAT') and os.path.exists('../OSAT'):
    sys.path.insert(1, os.path.abspath('..'))


from OSAT import thermometers
from OSAT.tools import vec_mc

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df_ol = pd.read_excel("input_ol.xlsx")
df_sp = pd.read_excel("input_sp.xlsx")
df_ol_err =  pd.read_excel("input_ol_err.xlsx")
df_sp_err = pd.read_excel("input_sp_err.xlsx")

In [4]:
mc_col_sp = df_sp_err.columns[1::]
mc_col_sp

Index(['SiO2', 'TiO2', 'Al2O3', 'Cr2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'NiO',
       'ZnO'],
      dtype='object')

In [5]:
mc_col_ol = df_ol_err.columns[1:-1]
mc_col_ol

Index(['SiO2', 'TiO2', 'Al2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'P2O5',
       'Cr2O3'],
      dtype='object')

In [6]:
mc = 1000

In [7]:
sp_mc_collect = pd.DataFrame()
for idx in range(len(df_sp)):
    df_iter = vec_mc(df_sp[mc_col_sp], df_sp_err[mc_col_sp], idx, mc)
    sp_mc_collect = sp_mc_collect.append(df_iter)

In [8]:
ol_mc_collect = pd.DataFrame()
for idx in range(len(df_ol)):
    df_iter = vec_mc(df_ol[mc_col_ol], df_ol_err[mc_col_ol], idx, mc)
    ol_mc_collect = ol_mc_collect.append(df_iter)

In [9]:
sp_mc_collect.columns = mc_col_sp
sp_mc_collect[sp_mc_collect < 0] = 0

In [10]:
ol_mc_collect.columns = mc_col_ol
ol_mc_collect[ol_mc_collect < 0] = 0

In [11]:
sp_mc_collect['sample_group'] = df_sp.loc[df_sp.index.repeat(mc)]['sample_no']  # avoid merge error in the script of alolivthermo
ol_mc_collect['sample_group'] = df_ol.loc[df_ol.index.repeat(mc)]['sample_no']

In [12]:
ol_mc_collect.reset_index(drop=True, inplace=True)
sp_mc_collect.reset_index(drop=True, inplace=True)

In [13]:
ol_mc_collect['sample_no'] = ol_mc_collect.index  # sampel_no match use
sp_mc_collect['sample_no'] = sp_mc_collect.index

In [14]:
reg = thermometers.models(ol_mc_collect, sp_mc_collect).compute()

In [15]:
reg.head()

,SiO2_ol,TiO2_ol,Al2O3_ol,FeO_ol,MnO_ol,MgO_ol,CaO_ol,Na2O_ol,P2O5_ol,Cr2O3_ol,sample_group_ol,sample_no,SiO2,TiO2,Al2O3,Cr2O3,FeO,MnO,MgO,CaO,NiO,ZnO,sample_group,SiO2_xca,TiO2_xca,Al2O3_xca,Cr2O3_xca,FeO_xca,MnO_xca,MgO_xca,sumcat_xca,sumchg_xca,fe3_xca,fe2_xca,fTet_xca,fOct_xca,proj_xca,mFeCr2O4_xca,mFe/MnAl2O4_xca,mFe3O4_xca,mMgAl2O4_xca,mFe2TiO4_xca,sum_comps_xca,X2_xca,X3_xca,X4_xca,X5_xca,reg_X2(1+X4-X2),reg_(1-X2)X3,reg_(1-X2)X4,reg_(1-X2)X5,reg_X3(X3+X4+X5),reg_X4(X3+X4+X5),reg_X5(X3+X4+X5),reg_X3X4,reg_X3X5,reg_X4X5,reg_X2pow0.5,reg_X2pow2,reg_X2,Cr#,lnkdal,lnkdcr,Fo,t_thermo,t_kdcr,t_kdal,t_coogan,t_zThermoAlCr,err_zThermoAlCr,t_zThermoAl,err_zThermoAl,z_thermo_cr,z_thermo_al,p_thermo_cr,p_thermo_al
0,40.177871,0.020157,0.032079,17.298350,0.270463,42.199430,0.240950,0.0,0.0,0.034374,PI082_02,0,0.077824,2.379622,25.383512,24.659676,32.185945,0.258528,11.564684,0.006734,0.0,0.0,PI082_02,0.001295,0.029795,0.497905,0.324490,0.447995,0.003644,0.286934,1.590763,3.167522,0.178523,0.269472,1,1,1.0,0.162245,-0.037981,0.089262,0.286934,0.029795,0.530254,0.541124,0.305976,0.056190,0.168338,0.236351,0.140405,0.025784,0.077246,0.162321,0.029809,0.089304,0.017193,0.051507,0.009459,0.735611,0.292815,0.541124,0.394565,-6.673661,-6.575627,0.813032,1092.386333,1114.939025,1162.355086,1174.013561,1162.355086,43.3,1162.355086,43.3,0.540527,1.414709,0.294417,0.078577
1,40.076181,0.018947,0.035385,17.419173,0.268417,42.272250,0.242936,0.0,0.0,0.055900,PI082_02,1,0.073865,2.268687,25.662549,26.678104,31.269106,0.253115,11.526393,0.004293,0.0,0.0,PI082_02,0.001229,0.028406,0.503378,0.351050,0.435234,0.003568,0.285983,1.607620,3.256013,0.160506,0.274728,1,1,1.0,0.175525,-0.034294,0.080253,0.285983,0.028406,0.535873,0.533677,0.327549,0.053009,0.149761,0.242176,0.152744,0.024719,0.069837,0.173706,0.028112,0.079421,0.017363,0.049054,0.007939,0.730532,0.284812,0.533677,0.410857,-6.586513,-6.168028,0.812235,1091.425781,1155.967806,1174.152378,1187.493118,1174.152378,43.3,1174.152378,43.3,1.546899,1.672661,0.060944,0.047197
2,40.141147,0.020768,0.036110,16.260474,0.265925,42.334829,0.244926,0.0,0.0,0.052040,PI082_02,2,0.069494,2.447465,25.138077,25.913991,32.666161,0.257179,11.440383,0.006421,0.0,0.0,PI082_02,0.001157,0.030645,0.493091,0.340995,0.454679,0.003625,0.283849,1.606885,3.199786,0.175881,0.278798,1,1,1.0,0.170498,-0.037304,0.087941,0.283849,0.030645,0.535628,0.529937,0.318313,0.057213,0.164182,0.247852,0.149627,0.026894,0.077176,0.171796,0.030878,0.088610,0.018212,0.052261,0.009393,0.727968,0.280834,0.529937,0.408823,-6.545571,-6.210532,0.822723,1097.978312,1158.319217,1182.104967,1195.758495,1182.104967,43.3,1182.104967,43.3,1.446209,1.700969,0.074059,0.044474
3,40.145132,0.019521,0.037160,16.157893,0.266331,42.493241,0.241150,0.0,0.0,0.018574,PI082_02,3,0.074715,2.164336,25.449296,24.400116,31.935468,0.255111,11.487313,0.004933,0.0,0.0,PI082_02,0.001244,0.027100,0.499195,0.321074,0.444509,0.003596,0.285014,1.580488,3.146428,0.179190,0.265319,1,1,1.0,0.160537,-0.035416,0.089595,0.285014,0.027100,0.526829,0.540998,0.304723,0.051439,0.170064,0.234293,0.139869,0.023611,0.078060,0.160354,0.027069,0.089492,0.015675,0.051823,0.008748,0.735526,0.292679,0.540998,0.391423,-6.529213,-7.180596,0.824186,1118.188879,1088.764716,1189.353445,1202.271945,1189.353445,43.3,1189.353445,43.3,0.705218,1.438887,0.240337,0.075091
4,39.985209,0.019270,0.035194,17.831903,0.268089,42.205946,0.240733,0.0,0.0,0.040228,PI082_02,4,0.077764,2.396225,25.409266,24.583513,32.282885,0.255048,11.595958,0.007653,0.0,0.0,PI082_02,0.001294,0.030003,0.498410,0.323488,0.449344,0.003595,0.287709,1.592550,3.168316,0.179796,0.269548,1,1,1.0,0.161744,-0.038504,0.089898,0.287709,0.030003,0.530850,0.541979,0.304688,0.056519,0.169347,0.235670,0.139554,0.025887,0.077565,0.161654,0.029986,0.089848,0.017221,0.051598,0.009571,0.736192,0.293741,0.541979,0.393584,-6.581980,-6.415257,0.808394,1106.581166,1139.071194,1179.129603,1191.629984,1179.129603,43.3,1179.129603,43.3,0.778699,1.46686

In [16]:
reg.groupby('sample_group')[['t_thermo', 't_kdcr', 't_kdal', 't_coogan','t_zThermoAlCr', 't_zThermoAl']].agg(['mean', 'std'])

t_thermo                 t_kdcr                t_kdal  \
                     mean       std         mean       std        mean   
sample_group                                                             
PI082_02      1090.726714  9.803521  1118.424513  29.90715  1171.19387   

                           t_coogan           t_zThermoAlCr             \
                   std         mean       std          mean        std   
sample_group                                                             
PI082_02      9.078991  1183.840263  9.530234   1167.910746  24.469676   

              t_zThermoAl             
                     mean        std  
sample_group                          
PI082_02      1170.805473  10.637597